**Homework 3- Question 3 Terrorist Problem Solution**

The given formulation for terrorist problem in the question is as follows

\begin{equation*}
\begin{aligned}
\min_{u} \quad \{\max_{x} & \{u^TAx\} \}\\
\textrm{s.t.} \quad & u \geq 0 \\
  & x \geq 0    \\
  & e^Tx = e^Tu = 1 \\
\end{aligned}
\end{equation*}

We can rewrite the above formulation in a way that is simple to code in Pyomo (by introducing a new variable). Here is an equivalent formulation to the above problem.

\begin{equation*}
\begin{aligned}
\min_{u,w} \quad  w\\
\textrm{s.t.} \quad & we^T -u^TA \geq 0 \\
  & u^Te = 1 \\
  & u \geq 0    \\
\end{aligned}
\end{equation*}

Now, let us code this in Pyomo


Importing Libraries

In [1]:
from pyomo.environ import *
import numpy as np

In [2]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 124947 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

Defining the model, sets and varibles

In [3]:
# Defining the Model
model = ConcreteModel()

# Sets
model.I = RangeSet(1, 3)

# Define variables
model.u = Var(model.I, domain=NonNegativeReals)
model.w = Var(domain = Reals)

Defining objective and constraints

In [4]:
# Objective
model.obj = Objective(expr= model.w, sense=minimize)

# Constraints
model.con1 = Constraint(expr=sum(model.u[j] for j in model.I) == 1)

# Define the matrix A as a dictionary
A_values = {(1, 1): 4, (1, 2): -10, (1, 3): -10, (2, 1): -8, (2, 2): 5, (2, 3): -8, (3, 1): -12, (3, 2): -12, (3, 3): 9}
model.A = Param(model.I, model.I, initialize=A_values)

def cost_constraint_rule(model, j):
    return model.w - sum(model.A[i,j] * model.u[i] for i in model.I) >= 0

model.con2 = Constraint(model.I, rule=cost_constraint_rule)

Soving the above model using a solver( I used Gurobi)

In [5]:
solver = SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': -4.59813084112149, 'Upper bound': -4.59813084112149, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 4, 'Number of nonzeros': 15, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.002846240997314453}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

Displaying the result

In [6]:
print("Optimal value of the objective function:", model.obj())
print("Optimal values of u:")
for j in model.I:
    print(f"u[{j}] =", value(model.u[j]))

Optimal value of the objective function: -4.59813084112149
Optimal values of u:
u[1] = 0.364485981308411
u[2] = 0.392523364485981
u[3] = 0.242990654205608
